# Example to understand the Write and Read lifecycle of Iceberg Query 

# 1. Insert query to understand write lifecycle of Iceberg Table

In [6]:
NEW_YORK_TAXI="hms.data_db.new_york_taxi"

In [1]:
# reading data into spark df from source csv file

taxi_df = spark.read.option("header", True).csv("/home/iceberg/datasets/new_york_taxi/yellow_tripdata_2019-02.csv")
taxi_df.show(2)

+--------------------+---------------------+---------------+-------------+------------+------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|PULocationID|DOLocationID|fare_amount|
+--------------------+---------------------+---------------+-------------+------------+------------+-----------+
|    01-02-2019 00:21|     01-02-2019 00:28|              1|          1.3|         163|         229|        6.5|
|    01-02-2019 00:32|     01-02-2019 00:45|              1|          3.7|         229|           7|       13.5|
+--------------------+---------------------+---------------+-------------+------------+------------+-----------+
only showing top 2 rows



In [2]:
# cleanup and transformation of df

from pyspark.sql.functions import col, to_timestamp, round

# Convert the datetime columns and passenger_count column to the appropriate data types
taxi_df = (
    taxi_df.withColumn("tpep_pickup_datetime", to_timestamp("tpep_pickup_datetime", "dd-MM-yyyy HH:mm"))
      .withColumn("tpep_dropoff_datetime", to_timestamp("tpep_dropoff_datetime", "dd-MM-yyyy HH:mm"))
      .withColumn("passenger_count", col("passenger_count").cast("BIGINT"))
      .withColumn("trip_distance", round(col("trip_distance"),2))
      .withColumn("trip_distance", col("trip_distance").cast("decimal(10,2)"))
      .withColumn("PULocationID", col("PULocationID").cast("BIGINT"))
      .withColumn("DOLocationID", col("DOLocationID").cast("BIGINT"))
      .withColumn("fare_amount", col("fare_amount").cast("FLOAT"))
)

In [3]:
taxi_df.createOrReplaceTempView("temp_taxi_feb2019")

In [4]:
spark.sql("INSERT INTO hms.data_db.new_york_taxi SELECT * FROM temp_taxi_feb2019")

DataFrame[]

In [7]:
spark.read.table(NEW_YORK_TAXI).show(5)

+--------------------+---------------------+---------------+-------------+------------+------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|PULocationID|DOLocationID|fare_amount|
+--------------------+---------------------+---------------+-------------+------------+------------+-----------+
| 2019-02-01 00:57:00|  2019-02-01 01:09:00|              2|         2.10|         141|         234|       10.0|
| 2019-02-01 00:21:00|  2019-02-01 00:28:00|              1|         1.30|         163|         229|        6.5|
| 2019-02-01 00:23:00|  2019-02-01 00:25:00|              1|         0.38|         170|         170|        3.5|
| 2019-02-01 00:39:00|  2019-02-01 00:48:00|              1|         0.55|         170|         170|        6.5|
| 2019-02-01 00:32:00|  2019-02-01 00:45:00|              1|         3.70|         229|           7|       13.5|
+--------------------+---------------------+---------------+-------------+------------+---------

In [11]:
%%sql


select count(1) from hms.data_db.new_york_taxi where tpep_pickup_datetime >= timestamp '2019-02-01 00:00:00' limit 5;

count(1)
11


In [1]:
%%sql

select * from hms.data_db.new_york_taxi.snapshots; 

24/11/22 05:38:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2024-11-20 12:16:20.695000,7915680060403489000,None,append,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/snap-7915680060403489000-1-d6087afd-1f0b-4d26-9e7b-5efb3bdf4c84.avro,"{'spark.app.id': 'local-1732103217067', 'changed-partition-count': '15', 'added-data-files': '16', 'total-equality-deletes': '0', 'added-records': '1048575', 'total-position-deletes': '0', 'added-files-size': '5901990', 'total-delete-files': '0', 'total-files-size': '5901990', 'total-records': '1048575', 'total-data-files': '16'}"
2024-11-22 05:11:42.701000,6118885366905035378,7915680060403489000,append,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/snap-6118885366905035378-1-82e22d0f-c4c8-4ed2-9ea7-98dbc40c8fe6.avro,"{'spark.app.id': 'local-1732252219747', 'changed-partition-count': '3', 'added-data-files': '3', 'total-equality-deletes': '0', 'added-records': '10', 'total-position-deletes': '0', 'added-files-size': '7016', 'total-delete-files': '0', 'total-files-size': '5909006', 'total-records': '1048585', 'total-data-files': '19'}"


In [2]:
%%sql

select * from hms.data_db.new_york_taxi.manifests; 

content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/82e22d0f-c4c8-4ed2-9ea7-98dbc40c8fe6-m0.avro,7624,0,6118885366905035378,3,0,0,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2019-02-01', upper_bound='2019-02-03')]"
0,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/d6087afd-1f0b-4d26-9e7b-5efb3bdf4c84-m0.avro,8798,0,7915680060403489000,16,0,0,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2003-01-01', upper_bound='2019-02-13')]"


In [11]:
%%sql

select * from hms.data_db.new_york_taxi.history; 

made_current_at,snapshot_id,parent_id,is_current_ancestor
2024-11-20 12:16:20.695000,7915680060403489000,None,True


In [4]:
%%sql

describe table extended hms.data_db.new_york_taxi;

col_name,data_type,comment
tpep_pickup_datetime,timestamp,None
tpep_dropoff_datetime,timestamp,None
passenger_count,bigint,None
trip_distance,"decimal(10,2)",None
PULocationID,bigint,None
DOLocationID,bigint,None
fare_amount,float,None
,,
# Partitioning,,
Part 0,days(tpep_pickup_datetime),


In [3]:
%%sql

select * from hms.data_db.new_york_taxi.metadata_log_entries;

timestamp,file,latest_snapshot_id,latest_schema_id,latest_sequence_number
2024-11-14 14:34:42.774000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00000-8baccb32-4a46-42ad-a9e2-aae45fa15c16.metadata.json,None,None,None
2024-11-20 12:15:47.503000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00001-f26e1a62-3c31-4a77-82ef-bacf6ab54050.metadata.json,None,None,None
2024-11-20 12:16:20.695000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00002-4aee5dc1-fdc6-413f-9363-b407e9ac41a7.metadata.json,7915680060403489000,0,1
2024-11-22 05:11:42.701000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00003-e6da6690-093f-4026-9443-77cc4738d38e.metadata.json,6118885366905035378,0,2


In [2]:

spark.sql( "show tblproperties hms.data_db.new_york_taxi").show();

+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
| current-snapshot-id| 6118885366905035378|
|              format|     iceberg/parquet|
|      format-version|                   2|
|          sort-order|PULocationID ASC ...|
|write.distributio...|               range|
|write.parquet.com...|                zstd|
+--------------------+--------------------+



In [6]:
%%sql
    
select * from hms.data_db.new_york_taxi.partitions order by last_updated_at desc, partition desc;

partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
"Row(tpep_pickup_datetime_day=datetime.date(2019, 2, 3))",0,2,1,2272,0,0,0,0,2024-11-22 05:11:42.701000,6118885366905035378
"Row(tpep_pickup_datetime_day=datetime.date(2019, 2, 2))",0,2,1,2272,0,0,0,0,2024-11-22 05:11:42.701000,6118885366905035378
"Row(tpep_pickup_datetime_day=datetime.date(2019, 2, 1))",0,6,1,2472,0,0,0,0,2024-11-22 05:11:42.701000,6118885366905035378
"Row(tpep_pickup_datetime_day=datetime.date(2019, 2, 13))",0,1,1,2206,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 6))",0,14,1,2716,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 5))",0,203374,1,1125553,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 4))",0,235044,1,1304710,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 3))",0,222879,2,1264862,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 2))",0,197852,1,1107912,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 1))",0,189035,1,1069566,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000


# 2. Delete query to understand write lifecycle of Iceberg Table

In [2]:
%%sql

delete from hms.data_db.new_york_taxi where tpep_pickup_datetime >= timestamp '2019-02-13 00:00:00';

24/11/24 08:17:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [3]:
%%sql


select count(1) from hms.data_db.new_york_taxi where tpep_pickup_datetime >= timestamp '2019-02-13 00:00:00';

count(1)
0


In [4]:
%%sql

select * from hms.data_db.new_york_taxi.snapshots; 

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2024-11-20 12:16:20.695000,7915680060403489000,None,append,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/snap-7915680060403489000-1-d6087afd-1f0b-4d26-9e7b-5efb3bdf4c84.avro,"{'spark.app.id': 'local-1732103217067', 'changed-partition-count': '15', 'added-data-files': '16', 'total-equality-deletes': '0', 'added-records': '1048575', 'total-position-deletes': '0', 'added-files-size': '5901990', 'total-delete-files': '0', 'total-files-size': '5901990', 'total-records': '1048575', 'total-data-files': '16'}"
2024-11-22 05:11:42.701000,6118885366905035378,7915680060403489000,append,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/snap-6118885366905035378-1-82e22d0f-c4c8-4ed2-9ea7-98dbc40c8fe6.avro,"{'spark.app.id': 'local-1732252219747', 'changed-partition-count': '3', 'added-data-files': '3', 'total-equality-deletes': '0', 'added-records': '10', 'total-position-deletes': '0', 'added-files-size': '7016', 'total-delete-files': '0', 'total-files-size': '5909006', 'total-records': '1048585', 'total-data-files': '19'}"
2024-11-24 08:17:59.515000,3733467730074381361,6118885366905035378,delete,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/snap-3733467730074381361-1-abbed668-2863-4c9b-9278-897cc1b5d5b3.avro,"{'spark.app.id': 'local-1732436070168', 'removed-files-size': '2206', 'changed-partition-count': '1', 'total-equality-deletes': '0', 'deleted-data-files': '1', 'total-position-deletes': '0', 'total-delete-files': '0', 'deleted-records': '1', 'total-files-size': '5906800', 'total-records': '1048584', 'total-data-files': '18'}"


In [5]:
%%sql

select * from hms.data_db.new_york_taxi.manifests; 

content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/82e22d0f-c4c8-4ed2-9ea7-98dbc40c8fe6-m0.avro,7624,0,6118885366905035378,3,0,0,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2019-02-01', upper_bound='2019-02-03')]"
0,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/abbed668-2863-4c9b-9278-897cc1b5d5b3-m0.avro,8813,0,3733467730074381361,0,15,1,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2003-01-01', upper_bound='2019-02-13')]"


In [6]:
%%sql

select * from hms.data_db.new_york_taxi.history; 

made_current_at,snapshot_id,parent_id,is_current_ancestor
2024-11-20 12:16:20.695000,7915680060403489000,None,True
2024-11-22 05:11:42.701000,6118885366905035378,7915680060403489000,True
2024-11-24 08:17:59.515000,3733467730074381361,6118885366905035378,True


In [7]:
%%sql

describe table extended hms.data_db.new_york_taxi;

col_name,data_type,comment
tpep_pickup_datetime,timestamp,None
tpep_dropoff_datetime,timestamp,None
passenger_count,bigint,None
trip_distance,"decimal(10,2)",None
PULocationID,bigint,None
DOLocationID,bigint,None
fare_amount,float,None
,,
# Partitioning,,
Part 0,days(tpep_pickup_datetime),


In [8]:
%%sql

select * from hms.data_db.new_york_taxi.metadata_log_entries;

timestamp,file,latest_snapshot_id,latest_schema_id,latest_sequence_number
2024-11-14 14:34:42.774000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00000-8baccb32-4a46-42ad-a9e2-aae45fa15c16.metadata.json,None,None,None
2024-11-20 12:15:47.503000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00001-f26e1a62-3c31-4a77-82ef-bacf6ab54050.metadata.json,None,None,None
2024-11-20 12:16:20.695000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00002-4aee5dc1-fdc6-413f-9363-b407e9ac41a7.metadata.json,7915680060403489000,0,1
2024-11-22 05:11:42.701000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00003-e6da6690-093f-4026-9443-77cc4738d38e.metadata.json,6118885366905035378,0,2
2024-11-24 08:17:59.515000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00004-ef478216-63af-4e54-b497-342a08322aae.metadata.json,3733467730074381361,0,3


In [9]:

spark.sql( "show tblproperties hms.data_db.new_york_taxi").show();

+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
| current-snapshot-id| 3733467730074381361|
|              format|     iceberg/parquet|
|      format-version|                   2|
|          sort-order|PULocationID ASC ...|
|write.distributio...|               range|
|write.parquet.com...|                zstd|
+--------------------+--------------------+



In [10]:
%%sql
    
select * from hms.data_db.new_york_taxi.partitions order by last_updated_at desc, partition desc;

partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
"Row(tpep_pickup_datetime_day=datetime.date(2019, 2, 3))",0,2,1,2272,0,0,0,0,2024-11-22 05:11:42.701000,6118885366905035378
"Row(tpep_pickup_datetime_day=datetime.date(2019, 2, 2))",0,2,1,2272,0,0,0,0,2024-11-22 05:11:42.701000,6118885366905035378
"Row(tpep_pickup_datetime_day=datetime.date(2019, 2, 1))",0,6,1,2472,0,0,0,0,2024-11-22 05:11:42.701000,6118885366905035378
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 6))",0,14,1,2716,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 5))",0,203374,1,1125553,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 4))",0,235044,1,1304710,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 3))",0,222879,2,1264862,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 2))",0,197852,1,1107912,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 1))",0,189035,1,1069566,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000
"Row(tpep_pickup_datetime_day=datetime.date(2018, 12, 31))",0,345,1,7784,0,0,0,0,2024-11-20 12:16:20.695000,7915680060403489000


# 3. Select query to understand read lifecycle of Iceberg Table

## Query with filter on partitioned column tpep_pickup_datetime

In [3]:
spark.sparkContext.setLogLevel("INFO")
spark.conf.set("spark.sql.iceberg.handle-timestamps", "true")

In [4]:
%%sql

SELECT * 
FROM hms.data_db.new_york_taxi 
WHERE tpep_pickup_datetime >= timestamp '2019-01-04 00:00:00' AND tpep_pickup_datetime < timestamp '2019-01-05 00:00:00' limit 10;


24/11/28 09:31:57 INFO BaseMetastoreTableOperations: Refreshing table metadata from new version: s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00004-ef478216-63af-4e54-b497-342a08322aae.metadata.json
24/11/28 09:31:57 INFO BaseMetastoreCatalog: Table loaded by catalog: hms.data_db.new_york_taxi
24/11/28 09:31:58 INFO SparkScanBuilder: Evaluating completely on Iceberg side: tpep_pickup_datetime IS NOT NULL
24/11/28 09:31:58 INFO SparkScanBuilder: Evaluating completely on Iceberg side: tpep_pickup_datetime >= 1546560000000000
24/11/28 09:31:58 INFO SparkScanBuilder: Evaluating completely on Iceberg side: tpep_pickup_datetime < 1546646400000000
24/11/28 09:31:58 INFO V2ScanRelationPushDown: 
Pushing operators to hms.data_db.new_york_taxi
Pushed Filters: tpep_pickup_datetime IS NOT NULL, tpep_pickup_datetime >= 1546560000000000, tpep_pickup_datetime < 1546646400000000
Post-Scan Filters: 
         
24/11/28 09:31:58 INFO V2ScanRelationPushDown: 
Output: tpep_pickup_datetime#52, t

tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,fare_amount
2019-01-04 00:37:00,2019-01-04 00:37:00,1,0.00,1,1,11.5
2019-01-04 05:55:00,2019-01-04 05:55:00,4,0.00,1,1,95.0
2019-01-04 06:33:00,2019-01-04 06:33:00,2,0.00,1,1,95.0
2019-01-04 06:54:00,2019-01-04 06:54:00,2,0.05,1,1,85.0
2019-01-04 11:14:00,2019-01-04 11:15:00,2,0.00,1,1,83.0
2019-01-04 13:04:00,2019-01-04 13:04:00,1,0.10,1,1,85.0
2019-01-04 15:28:00,2019-01-04 15:29:00,1,0.01,1,1,95.0
2019-01-04 15:04:00,2019-01-04 15:04:00,5,0.00,1,1,102.30000305175781
2019-01-04 15:33:00,2019-01-04 15:33:00,3,0.00,1,1,100.0
2019-01-04 16:14:00,2019-01-04 16:14:00,1,0.00,1,1,100.0


In [6]:
%%sql
    
explain cost
SELECT * 
FROM hms.data_db.new_york_taxi 
WHERE tpep_pickup_datetime BETWEEN timestamp '2019-02-01 00:00:00' AND timestamp '2019-02-03 23:59:59';

24/11/28 09:34:04 INFO BaseMetastoreTableOperations: Refreshing table metadata from new version: s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00004-ef478216-63af-4e54-b497-342a08322aae.metadata.json
24/11/28 09:34:04 INFO BaseMetastoreCatalog: Table loaded by catalog: hms.data_db.new_york_taxi
24/11/28 09:34:04 INFO SparkScanBuilder: Evaluating completely on Iceberg side: tpep_pickup_datetime IS NOT NULL
24/11/28 09:34:04 INFO SparkScanBuilder: Evaluating completely on Iceberg side: tpep_pickup_datetime >= 1548979200000000
24/11/28 09:34:04 INFO V2ScanRelationPushDown: 
Pushing operators to hms.data_db.new_york_taxi
Pushed Filters: tpep_pickup_datetime IS NOT NULL, tpep_pickup_datetime >= 1548979200000000, tpep_pickup_datetime <= 1549238399000000
Post-Scan Filters: (tpep_pickup_datetime#107 <= 2019-02-03 23:59:59)
         
24/11/28 09:34:04 INFO V2ScanRelationPushDown: 
Output: tpep_pickup_datetime#107, tpep_dropoff_datetime#108, passenger_count#109L, trip_distance#110, PU

plan
"== Optimized Logical Plan ==Filter (tpep_pickup_datetime#107 <= 2019-02-03 23:59:59), Statistics(sizeInBytes=520.0 B)+- RelationV2[tpep_pickup_datetime#107, tpep_dropoff_datetime#108, passenger_count#109L, trip_distance#110, PULocationID#111L, DOLocationID#112L, fare_amount#113] hms.data_db.new_york_taxi, Statistics(sizeInBytes=520.0 B, rowCount=10)== Physical Plan ==*(1) Filter (tpep_pickup_datetime#107 <= 2019-02-03 23:59:59)+- *(1) ColumnarToRow +- BatchScan hms.data_db.new_york_taxi[tpep_pickup_datetime#107, tpep_dropoff_datetime#108, passenger_count#109L, trip_distance#110, PULocationID#111L, DOLocationID#112L, fare_amount#113] hms.data_db.new_york_taxi (branch=null) [filters=tpep_pickup_datetime IS NOT NULL, tpep_pickup_datetime >= 1548979200000000, tpep_pickup_datetime <= 1549238399000000, groupedBy=] RuntimeFilters: []"


24/11/28 09:34:04 INFO BlockManagerInfo: Removed broadcast_6_piece0 on spark:42989 in memory (size: 3.6 KiB, free: 434.4 MiB)
24/11/28 09:34:04 INFO BlockManagerInfo: Removed broadcast_3_piece0 on spark:42989 in memory (size: 3.6 KiB, free: 434.4 MiB)
24/11/28 09:34:04 INFO BlockManagerInfo: Removed broadcast_10_piece0 on spark:42989 in memory (size: 3.6 KiB, free: 434.4 MiB)
24/11/28 09:34:04 INFO BlockManagerInfo: Removed broadcast_4_piece0 on spark:42989 in memory (size: 3.6 KiB, free: 434.4 MiB)
24/11/28 09:34:04 INFO BlockManagerInfo: Removed broadcast_1_piece0 on spark:42989 in memory (size: 3.6 KiB, free: 434.4 MiB)
24/11/28 09:34:04 INFO BlockManagerInfo: Removed broadcast_8_piece0 on spark:42989 in memory (size: 6.0 KiB, free: 434.4 MiB)
24/11/28 09:34:04 INFO BlockManagerInfo: Removed broadcast_7_piece0 on spark:42989 in memory (size: 3.6 KiB, free: 434.4 MiB)


## Query with filter on non-partitioned column tpep_dropoff_datetime

In [7]:
%%sql

SELECT * 
FROM hms.data_db.new_york_taxi 
WHERE tpep_dropoff_datetime >= timestamp '2019-01-04 00:00:00' AND tpep_dropoff_datetime < timestamp '2019-01-05 00:00:00' limit 10;

24/11/28 09:35:15 INFO BaseMetastoreTableOperations: Refreshing table metadata from new version: s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00004-ef478216-63af-4e54-b497-342a08322aae.metadata.json
24/11/28 09:35:15 INFO BaseMetastoreCatalog: Table loaded by catalog: hms.data_db.new_york_taxi
24/11/28 09:35:15 INFO V2ScanRelationPushDown: 
Pushing operators to hms.data_db.new_york_taxi
Pushed Filters: tpep_dropoff_datetime IS NOT NULL, tpep_dropoff_datetime >= 1546560000000000, tpep_dropoff_datetime < 1546646400000000
Post-Scan Filters: isnotnull(tpep_dropoff_datetime#127),(tpep_dropoff_datetime#127 >= 2019-01-04 00:00:00),(tpep_dropoff_datetime#127 < 2019-01-05 00:00:00)
         
24/11/28 09:35:15 INFO V2ScanRelationPushDown: 
Output: tpep_pickup_datetime#126, tpep_dropoff_datetime#127, passenger_count#128L, trip_distance#129, PULocationID#130L, DOLocationID#131L, fare_amount#132
         
24/11/28 09:35:15 INFO SnapshotScan: Scanning table hms.data_db.new_york_taxi snap

tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,fare_amount
2019-01-03 23:56:00,2019-01-04 00:09:00,2,5.13,4,40,16.0
2019-01-03 23:56:00,2019-01-04 00:06:00,1,2.08,4,231,9.5
2019-01-03 12:17:00,2019-01-04 11:36:00,5,3.64,4,246,15.0
2019-01-03 23:59:00,2019-01-04 00:02:00,1,0.70,7,7,4.5
2019-01-03 23:50:00,2019-01-04 00:15:00,5,4.66,7,7,22.0
2019-01-03 20:22:00,2019-01-04 18:07:00,1,15.90,10,140,52.0
2019-01-03 15:21:00,2019-01-04 14:54:00,1,13.48,10,164,52.0
2019-01-03 23:38:00,2019-01-04 00:10:00,1,17.22,10,238,52.0
2019-01-03 23:59:00,2019-01-04 00:11:00,1,4.70,13,25,15.5
2019-01-03 23:51:00,2019-01-04 00:01:00,1,2.04,13,45,9.5


In [8]:
%%sql

explain  COST
SELECT * 
FROM hms.data_db.new_york_taxi 
WHERE tpep_dropoff_datetime >= timestamp '2019-01-04 00:00:00' AND tpep_dropoff_datetime < timestamp '2019-01-05 00:00:00';

24/11/28 09:35:25 INFO V2ScanRelationPushDown: 
Pushing operators to hms.data_db.new_york_taxi
Pushed Filters: tpep_dropoff_datetime IS NOT NULL, tpep_dropoff_datetime >= 1546560000000000, tpep_dropoff_datetime < 1546646400000000
Post-Scan Filters: isnotnull(tpep_dropoff_datetime#158),(tpep_dropoff_datetime#158 >= 2019-01-04 00:00:00),(tpep_dropoff_datetime#158 < 2019-01-05 00:00:00)
         
24/11/28 09:35:25 INFO V2ScanRelationPushDown: 
Output: tpep_pickup_datetime#157, tpep_dropoff_datetime#158, passenger_count#159L, trip_distance#160, PULocationID#161L, DOLocationID#162L, fare_amount#163
         
24/11/28 09:35:25 INFO SnapshotScan: Scanning table hms.data_db.new_york_taxi snapshot 3733467730074381361 created at 2024-11-24T08:17:59.515+00:00 with filter ((tpep_dropoff_datetime IS NOT NULL AND tpep_dropoff_datetime >= (16-digit-int)) AND tpep_dropoff_datetime < (16-digit-int))
24/11/28 09:35:25 INFO BaseDistributedDataScan: Planning file tasks locally for table hms.data_db.new_yo

plan
"== Optimized Logical Plan ==Filter ((isnotnull(tpep_dropoff_datetime#158) AND (tpep_dropoff_datetime#158 >= 2019-01-04 00:00:00)) AND (tpep_dropoff_datetime#158 < 2019-01-05 00:00:00)), Statistics(sizeInBytes=32.1 MiB)+- RelationV2[tpep_pickup_datetime#157, tpep_dropoff_datetime#158, passenger_count#159L, trip_distance#160, PULocationID#161L, DOLocationID#162L, fare_amount#163] hms.data_db.new_york_taxi, Statistics(sizeInBytes=32.1 MiB, rowCount=6.47E+5)== Physical Plan ==*(1) Filter ((isnotnull(tpep_dropoff_datetime#158) AND (tpep_dropoff_datetime#158 >= 2019-01-04 00:00:00)) AND (tpep_dropoff_datetime#158 < 2019-01-05 00:00:00))+- *(1) ColumnarToRow +- BatchScan hms.data_db.new_york_taxi[tpep_pickup_datetime#157, tpep_dropoff_datetime#158, passenger_count#159L, trip_distance#160, PULocationID#161L, DOLocationID#162L, fare_amount#163] hms.data_db.new_york_taxi (branch=null) [filters=tpep_dropoff_datetime IS NOT NULL, tpep_dropoff_datetime >= 1546560000000000, tpep_dropoff_datetime < 1546646400000000, groupedBy=] RuntimeFilters: []"


In [10]:
%%sql
    
EXPLAIN FORMATTED 
SELECT * FROM hms.data_db.new_york_taxi 
WHERE tpep_pickup_datetime BETWEEN timestamp '2019-02-01 00:00:00' AND timestamp '2019-02-03 23:59:59';

24/11/28 09:36:10 INFO BaseMetastoreTableOperations: Refreshing table metadata from new version: s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00004-ef478216-63af-4e54-b497-342a08322aae.metadata.json
24/11/28 09:36:10 INFO BaseMetastoreCatalog: Table loaded by catalog: hms.data_db.new_york_taxi
24/11/28 09:36:10 INFO SparkScanBuilder: Evaluating completely on Iceberg side: tpep_pickup_datetime IS NOT NULL
24/11/28 09:36:10 INFO SparkScanBuilder: Evaluating completely on Iceberg side: tpep_pickup_datetime >= 1548979200000000
24/11/28 09:36:10 INFO V2ScanRelationPushDown: 
Pushing operators to hms.data_db.new_york_taxi
Pushed Filters: tpep_pickup_datetime IS NOT NULL, tpep_pickup_datetime >= 1548979200000000, tpep_pickup_datetime <= 1549238399000000
Post-Scan Filters: (tpep_pickup_datetime#210 <= 2019-02-03 23:59:59)
         
24/11/28 09:36:10 INFO V2ScanRelationPushDown: 
Output: tpep_pickup_datetime#210, tpep_dropoff_datetime#211, passenger_count#212L, trip_distance#213, PU

plan
"== Physical Plan ==* Filter (3)+- * ColumnarToRow (2) +- BatchScan hms.data_db.new_york_taxi (1)(1) BatchScan hms.data_db.new_york_taxiOutput [7]: [tpep_pickup_datetime#210, tpep_dropoff_datetime#211, passenger_count#212L, trip_distance#213, PULocationID#214L, DOLocationID#215L, fare_amount#216]hms.data_db.new_york_taxi (branch=null) [filters=tpep_pickup_datetime IS NOT NULL, tpep_pickup_datetime >= 1548979200000000, tpep_pickup_datetime <= 1549238399000000, groupedBy=](2) ColumnarToRow [codegen id : 1]Input [7]: [tpep_pickup_datetime#210, tpep_dropoff_datetime#211, passenger_count#212L, trip_distance#213, PULocationID#214L, DOLocationID#215L, fare_amount#216](3) Filter [codegen id : 1]Input [7]: [tpep_pickup_datetime#210, tpep_dropoff_datetime#211, passenger_count#212L, trip_distance#213, PULocationID#214L, DOLocationID#215L, fare_amount#216]Condition : (tpep_pickup_datetime#210 <= 2019-02-03 23:59:59)"
